In [1]:
import numpy as np
import pandas as pd
import netCDF4
from netCDF4 import Dataset
from netCDF4 import MFDataset
import datetime
from pathlib import Path
import xarray as xr
from datetime import datetime

In [2]:
era5_down_test = xr.open_dataset('hanle.nc')

In [3]:
era5_down_test = era5_down_test.to_dataframe()

In [4]:
era5_down_test.tail(10)

blh
latitude longitude time                          
32.75    79.0      1980-12-01 14:00:00  77.629639
                   1980-12-01 15:00:00  36.145996
                   1980-12-01 16:00:00  37.535400
                   1980-12-01 17:00:00  39.784912
                   1980-12-01 18:00:00  44.813232
                   1980-12-01 19:00:00  51.627930
                   1980-12-01 20:00:00  57.516357
                   1980-12-01 21:00:00  60.493652
                   1980-12-01 22:00:00  57.317871
                   1980-12-01 23:00:00  59.170410

In [5]:
#data_folder = Path('Project Data/3.ERA5/')
era = xr.open_mfdataset('D:/Codes/IIA Project/Project Data/3.ERA5/*Hanle.nc')
eram= xr.open_mfdataset('D:/Codes/IIA Project/Project Data/3.ERA5/*Merak.nc')


In [6]:
era_h = era.to_dataframe().reset_index()

In [7]:
era_m = eram.to_dataframe().reset_index()

In [8]:
erai5 = pd.DataFrame(columns = ['date','time','hanle_pblh_era5','merak_pblh_era5'])

In [9]:
erai5['time'] = era_h['time'].apply(lambda x:x.time())
erai5['date'] = era_h['time'].apply(lambda x:x.date())

In [10]:
erai5['hanle_pblh_era5'] = era_h['blh']/1000
erai5['merak_pblh_era5'] = era_m['blh']/1000
#erai5['hanle_pblh'] = np.ravel(era['blh'])
#erai5['merak_pblh'] = np.ravel(eram['blh'])

In [11]:
erai5.head(2)

,date,time,hanle_pblh_era5,merak_pblh_era5
0,1979-01-01,00:00:00,0.029268,0.037020
1,1979-01-01,01:00:00,0.025694,0.033325


In [20]:
#erai4 = era.to_dataframe()

#Comments by Aswin
# Why is this even required?
# You could have done extraction of various date components with erai5 itself right?
#erai4 = erai5

#erai4['Year'] = erai5.time.apply(lambda x: x.year)
#erai4['Month'] = erai5.time.apply(lambda x: x.month)
#erai4['Day'] = erai5.time.apply(lambda x: x.day)
#erai4['Hour'] = erai5.time.apply(lambda x: x.hour)
#apply function works only columnwise by default

#erai4 = erai4.drop('time',axis=1)
# Comment by Aswin
# You can just use the statement below
# del erai4["time"]

In [12]:
erai5.to_csv('D:/Codes/IIA Project/Output Data/erai5_full.txt',sep = '\t',index=False)

In [9]:
avg_daily = erai5[['date','hanle_pblh_era5','merak_pblh_era5']].groupby(['date']).agg('mean')
median_daily = erai5[['date','hanle_pblh_era5','merak_pblh_era5']].groupby(['date']).agg('median')

In [12]:
era5_daily = pd.merge(avg_daily,median_daily,left_index=True,right_index=True,suffixes=('_avg','_median'))

In [15]:
erai5_daytime = erai5[(erai5['time'] >= datetime.strptime('06:00:00','%H:%M:%S').time()) & (erai5['time'] < datetime.strptime('18:00:00','%H:%M:%S').time())]

In [13]:
era5_daily.head(2)

,hanle_pblh_era5_avg,merak_pblh_era5_avg,hanle_pblh_era5_median,merak_pblh_era5_median
date,,,,
1979-01-01,0.143104,0.143104,0.026773,0.026773
1979-01-02,0.274187,0.274187,0.028391,0.028391


In [14]:
erai5_nighttime = erai5[(erai5['time'] < datetime.strptime('06:00:00','%H:%M:%S').time()) | (erai5['time'] >= datetime.strptime('18:00:00','%H:%M:%S').time())]

In [16]:
avg_daytime = erai5_daytime.groupby('date').mean()
avg_nighttime = erai5_nighttime.groupby('date').mean()
median_daytime = erai5_daytime.groupby('date').median()
median_nighttime= erai5_nighttime.groupby('date').median()

In [17]:
era5_daytime = pd.merge(avg_daytime,median_daytime,left_index=True,right_index=True,suffixes=('_avg','_median'))
era5_nighttime = pd.merge(avg_nighttime,median_nighttime,left_index=True,right_index=True,suffixes=('_avg','_median'))

In [40]:
avg_daily.to_csv('D:/Codes/IIA Project/Output Data/era5_daily_avg.txt',sep='\t')
avg_daytime.to_csv('D:/Codes/IIA Project/Output Data/era5_daytime_avg.txt',sep='\t')
avg_nighttime.to_csv('D:/Codes/IIA Project/Output Data/era5_nighttime_avg.txt',sep='\t')

In [18]:
era5_daily.to_csv('D:/Codes/IIA Project/Output Data/era5_daily.txt',sep='\t')
era5_daytime.to_csv('D:/Codes/IIA Project/Output Data/era5_daytime.txt',sep='\t')
era5_nighttime.to_csv('D:/Codes/IIA Project/Output Data/era5_nighttime.txt',sep='\t')

In [19]:
erai5['year'] = erai5['date'].apply(lambda x:x.year)
erai5['month'] = erai5['date'].apply(lambda x:x.month)

In [22]:
avg_monthly = erai5.groupby(['year','month']).mean().reset_index([0,1])

In [20]:
median_monthly = erai5.groupby(['year','month']).median().reset_index([0,1])

In [23]:
era5_monthly = pd.merge(avg_monthly,median_monthly,how='inner',left_on=['year','month'],right_on=['year','month'],suffixes=('_avg','_median'))

In [50]:
avg_monthly.to_csv('D:/Codes/IIA Project/Output Data/era5_monthly_avg.txt',sep='\t',index=False)

In [25]:
era5_monthly.to_csv('D:/Codes/IIA Project/Output Data/era5_monthly.txt',sep='\t',index=False)

In [52]:
avg_matching_date = avg_daily[datetime.strptime('2011-11-14','%Y-%m-%d').date():datetime.strptime('2013-03-26', '%Y-%m-%d').date()]

In [54]:
seeing_data = pd.read_csv('merged_erai_seeing_han.csv', parse_dates = ['date'])

In [55]:
seeing_data['date'] = seeing_data['date'].apply(lambda x: x.date())
seeing_data.set_index('date',inplace=True)

In [61]:
merged = pd.merge(seeing_data,avg_daily,how='left',left_index=True,right_index=True)

In [59]:
merged.head(10)

,hanle_pblh,merak_pblh,PT,See1,See2,std,hanle_pblh_era5,merak_pblh_era5
date,,,,,,,,
2011-11-14,0.833101,916.589000,44,1.228,1.145,0.389,1.079512,1.079512
2011-11-15,0.783074,845.967650,1,1.060,1.060,0.000,0.982901,0.982901
2011-11-16,0.547636,578.767330,9,1.231,1.270,0.114,0.841915,0.841915
2011-11-18,0.417248,432.263200,193,1.208,1.200,0.186,1.026768,1.026768
2011-11-19,0.375089,186.438460,151,1.133,1.110,0.164,0.897806,0.897806
2011-11-20,0.378447,126.010650,259,1.106,1.090,0.148,0.754552,0.754552
2011-11-21,0.251490,49.458130,265,0.988,0.970,0.131,0.444922,0.444922
2011-11-22,0.270785,41.177948,258,1.336,1.330,0.403,0.464491,0.464491
2011-11-23,0.244197,46.024810,159,2.017,1.990,0.541,0.633236,0.633236


In [62]:
merged.plot( y = ['hanle_pblh','See2','hanle_pblh_era5'],figsize=(20,10))